In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import itertools
import warnings
warnings.filterwarnings("ignore")

##  Load and preprocess data

In [3]:
# Load and preprocess your data
file_path = 'Cambodia_Tourism_Statistics_2013_2024.csv'
data = pd.read_csv(file_path)
data

,Months,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Q1,"1,172,072","1,267,922","1,307,836","1,342,477","1,502,896","1,711,253","1,877,853","1,155,226","70,901","159,546","1,291,539","1,582,677"
1,Jan,"404,106","442,045","460,577","466,086","532,206","596,241","647,206","547,963","20,567","44,910","402,943","540,023"
2,Feb,"385,760","425,801","430,207","448,468","493,316","542,937","597,483","383,863","20,580","50,410","434,503","448,551"
3,Mar,"382,206","400,076","417,052","427,923","477,374","572,075","633,164","223,400","29,754","64,325","453,093","594,103"
4,Q2,"920,527","933,446","994,154","1,018,455","1,159,783","1,290,407","1,460,621","27,600","31,659","347,216","1,288,393","533,932"
5,Apr,"327,000","332,690","361,139","367,684","412,925","463,423","537,656","4,841","11,911","81,939","430,129","533,932"
6,May,"292,115","300,302","314,748","320,601","368,809","419,171","472,950","10,475","8,757","101,979","431,650","524,390"
7,Jun,"301,412","300,454","318,267","330,170","378,049","407,813","450,013","12,284","10,964","163,298","416,150",NaN
8,Q3,"964,612","998,690","1,044,880","1,147,483","1,250,082","1,374,373","1,475,823","64,854","29,144","759,010","1,346,950",NaN
9,Jul,"338,761","340,091","364,325","395,761","446,627","454,056","502,421","24,029","26,896","336,769","457,412",NaN


In [4]:
# Extract the monthly data
monthly_data = data.loc[data['Months'].isin(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])]
monthly_data

,Months,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,Jan,"404,106","442,045","460,577","466,086","532,206","596,241","647,206","547,963","20,567","44,910","402,943","540,023"
2,Feb,"385,760","425,801","430,207","448,468","493,316","542,937","597,483","383,863","20,580","50,410","434,503","448,551"
3,Mar,"382,206","400,076","417,052","427,923","477,374","572,075","633,164","223,400","29,754","64,325","453,093","594,103"
5,Apr,"327,000","332,690","361,139","367,684","412,925","463,423","537,656","4,841","11,911","81,939","430,129","533,932"
6,May,"292,115","300,302","314,748","320,601","368,809","419,171","472,950","10,475","8,757","101,979","431,650","524,390"
7,Jun,"301,412","300,454","318,267","330,170","378,049","407,813","450,013","12,284","10,964","163,298","416,150",NaN
9,Jul,"338,761","340,091","364,325","395,761","446,627","454,056","502,421","24,029","26,896","336,769","457,412",NaN
10,Aug,"342,064","347,211","366,096","406,214","427,224","494,043","519,502","23,028","19,463","254,813","464,637",NaN
11,Sep,"283,787","311,388","314,459","345,508","376,231","426,274","453,900","21,137","9,967","267,500","424,901",NaN
13,Oct,"334,410","390,637","408,922","414,077","417,039","453,370","481,782","20,210","12,759","310,182","480,330",NaN


In [5]:
# Reshape the data to have a 'ds' (date) and 'y' (value) format
reshaped_data = monthly_data.melt(id_vars='Months', var_name='Year', value_name='Arrivals')

# Convert 'Months' and 'Year' into a datetime format
reshaped_data['Year'] = reshaped_data['Year'].astype(int)
reshaped_data['ds'] = pd.to_datetime(reshaped_data['Months'] + ' ' + reshaped_data['Year'].astype(str))

# Clean up the arrivals data
reshaped_data['y'] = reshaped_data['Arrivals'].str.replace(',', '').astype(float)

# Filter out only the necessary columns
reshaped_data = reshaped_data[['ds', 'y']]

# Drop rows with NaN values
reshaped_data.dropna(inplace=True)

#Print reshaped data
reshaped_data

,ds,y
0,2013-01-01,404106.0
1,2013-02-01,385760.0
2,2013-03-01,382206.0
3,2013-04-01,327000.0
4,2013-05-01,292115.0
...,...,...
132,2024-01-01,540023.0
133,2024-02-01,448551.0
134,2024-03-01,594103.0
135,2024-04-01,533932.0


In [6]:
# Split the data into training and test sets
train = reshaped_data[reshaped_data['ds'] < '2023-01-01']
test = reshaped_data[reshaped_data['ds'] >= '2023-01-01']

# train test split length
print("Train data : ", train)
print("Test data : ", test)

Train data :              ds         y
0   2013-01-01  404106.0
1   2013-02-01  385760.0
2   2013-03-01  382206.0
3   2013-04-01  327000.0
4   2013-05-01  292115.0
..         ...       ...
115 2022-08-01  254813.0
116 2022-09-01  267500.0
117 2022-10-01  310182.0
118 2022-11-01  330181.0
119 2022-12-01  362571.0

[120 rows x 2 columns]
Test data :              ds         y
120 2023-01-01  402943.0
121 2023-02-01  434503.0
122 2023-03-01  453093.0
123 2023-04-01  430129.0
124 2023-05-01  431650.0
125 2023-06-01  416150.0
126 2023-07-01  457412.0
127 2023-08-01  464637.0
128 2023-09-01  424901.0
129 2023-10-01  480330.0
130 2023-11-01  510231.0
131 2023-12-01  535738.0
132 2024-01-01  540023.0
133 2024-02-01  448551.0
134 2024-03-01  594103.0
135 2024-04-01  533932.0
136 2024-05-01  524390.0


### ARIMA Model Training and Forecasting

In [7]:
# Function to perform grid search for ARIMA
def grid_search_arima(train_data, p_values, d_values, q_values, seasonal_pdq):
    best_score, best_cfg, best_seasonal_cfg = float("inf"), None, None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                for seasonal in seasonal_pdq:
                    try:
                        model = SARIMAX(train_data, order=(p, d, q), seasonal_order=seasonal)
                        model_fit = model.fit(disp=False)
                        yhat = model_fit.predict(len(train_data), len(train_data) + len(test) - 1)
                        mae = mean_absolute_error(test['y'], yhat)
                        if mae < best_score:
                            best_score, best_cfg, best_seasonal_cfg = mae, (p, d, q), seasonal
                        print(f'ARIMA{(p,d,q)}x{seasonal} MAE={mae}')
                    except:
                        continue
    return best_cfg, best_seasonal_cfg

# Perform grid search to find the best ARIMA parameters
p_values = range(0, 3)
d_values = range(0, 3)
q_values = range(0, 3)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(range(0, 3), repeat=3))]
best_cfg, best_seasonal_cfg = grid_search_arima(train['y'], p_values, d_values, q_values, seasonal_pdq)

# Train the best ARIMA model
arima_model = SARIMAX(train['y'], order=best_cfg, seasonal_order=best_seasonal_cfg)
arima_model_fit = arima_model.fit(disp=False)

# Make predictions with the best ARIMA model
arima_forecast = arima_model_fit.get_forecast(steps=len(test) + 6)
arima_forecast_mean = arima_forecast.predicted_mean
arima_forecast_ci = arima_forecast.conf_int(alpha=0.05)

# Extract the ARIMA future forecast
arima_future_forecast = arima_forecast_mean[len(test):]

# Evaluate the improved ARIMA model performance
arima_mae = mean_absolute_error(test['y'], arima_forecast_mean[:len(test)])
arima_rmse = np.sqrt(mean_squared_error(test['y'], arima_forecast_mean[:len(test)]))
arima_mape = np.mean(np.abs((test['y'] - arima_forecast_mean[:len(test)]) / test['y'])) * 100
arima_r2 = r2_score(test['y'], arima_forecast_mean[:len(test)])

print(f'Improved ARIMA MAE: {arima_mae}')
print(f'Improved ARIMA RMSE: {arima_rmse}')
print(f'Improved ARIMA MAPE: {arima_mape}')
print(f'Improved ARIMA R2: {arima_r2}')




ARIMA(0, 0, 0)x(0, 0, 0, 12) MAE=475453.8823529412
ARIMA(0, 0, 0)x(0, 0, 1, 12) MAE=416678.0339777184
ARIMA(0, 0, 0)x(0, 0, 2, 12) MAE=305309.41855098575
ARIMA(0, 0, 0)x(0, 1, 0, 12) MAE=315898.5882352941
ARIMA(0, 0, 0)x(0, 1, 1, 12) MAE=301024.182970934
ARIMA(0, 0, 0)x(0, 1, 2, 12) MAE=288791.30141218525
ARIMA(0, 0, 0)x(0, 2, 0, 12) MAE=247074.29411764705
ARIMA(0, 0, 0)x(0, 2, 1, 12) MAE=357480.8041455257
ARIMA(0, 0, 0)x(0, 2, 2, 12) MAE=290725.7057940798
ARIMA(0, 0, 0)x(1, 0, 0, 12) MAE=339272.6277807728
ARIMA(0, 0, 0)x(1, 0, 1, 12) MAE=316466.9510778821
ARIMA(0, 0, 0)x(1, 0, 2, 12) MAE=295484.68213403673
ARIMA(0, 0, 0)x(1, 1, 0, 12) MAE=309261.5472971802
ARIMA(0, 0, 0)x(1, 1, 1, 12) MAE=306890.2263316213
ARIMA(0, 0, 0)x(1, 1, 2, 12) MAE=246617.57421354268
ARIMA(0, 0, 0)x(1, 2, 0, 12) MAE=277312.2863808849
ARIMA(0, 0, 0)x(1, 2, 1, 12) MAE=306795.3255295544
ARIMA(0, 0, 0)x(1, 2, 2, 12) MAE=319682.3274496311
ARIMA(0, 0, 0)x(2, 0, 0, 12) MAE=331667.03394421557
ARIMA(0, 0, 0)x(2, 0, 1, 1

In [8]:
# Plot ARIMA results
fig = make_subplots(rows=1, cols=1)

# Training data
fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Training Data', line=dict(color='royalblue', width=2)))

# Test data
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Actual Data', line=dict(color='red', width=2)))

# ARIMA Forecast
fig.add_trace(go.Scatter(x=test['ds'], y=arima_forecast_mean[:len(test)], mode='lines', name='ARIMA Forecast', line=dict(dash='dash', color='green', width=2)))

# Customize layout
fig.update_layout(
    title={
        'text': 'ARIMA Model: Training, Test, and Forecast',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'font': dict(size=20, color='darkblue')
    },
    xaxis_title='Date',
    yaxis_title='Tourism Arrivals',
    yaxis=dict(
        tickformat='~s',  # Format y-axis labels to show "K" for thousands
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    xaxis=dict(
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    legend=dict(
        x=1.1,  # Position legend outside the chart
        y=1,
        xanchor='left',
        yanchor='top',
        traceorder='normal',
        font=dict(size=12),
        bgcolor='rgba(255, 255, 255, 0)'
    ),
    plot_bgcolor='white',  # Set plot background color
    paper_bgcolor='white',  # Set paper background color
    margin=dict(l=40, r=40, t=80, b=40)  # Adjust margins
)

fig.show()


###  XGBoost Model Training and Forecasting

In [9]:
# Additional feature engineering for XGBoost
reshaped_data['lag_1'] = reshaped_data['y'].shift(1)
reshaped_data['rolling_mean_3'] = reshaped_data['y'].rolling(window=3).mean()
reshaped_data['rolling_std_3'] = reshaped_data['y'].rolling(window=3).std()
reshaped_data['rolling_mean_6'] = reshaped_data['y'].rolling(window=6).mean()
reshaped_data['rolling_std_6'] = reshaped_data['y'].rolling(window=6).std()

# Drop rows with NaN values after creating new features
reshaped_data.dropna(inplace=True)

# Split the data into training and test sets with new features
train = reshaped_data[reshaped_data['ds'] < '2023-01-01']
test = reshaped_data[reshaped_data['ds'] >= '2023-01-01']

# Separate features and target variable with new features
X_train = train.drop(columns=['ds', 'y'])
y_train = train['y']
X_test = test.drop(columns=['ds', 'y'])
y_test = test['y']

# Define the parameter grid for XGBoost with RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}

# Initialize the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Perform randomized search for XGBoost
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=100, scoring='neg_mean_absolute_error', cv=3, verbose=2, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Print best parameters for XGBoost
print(f'Best parameters: {random_search.best_params_}')

# Train the best XGBoost model
best_xgb_model = random_search.best_estimator_
best_xgb_model.fit(X_train, y_train)

# Make predictions with XGBoost
xgb_forecast = best_xgb_model.predict(X_test)

# Function to create future features dynamically
def create_future_features(last_known_values, num_future_periods):
    future_features = []
    last_known_values = last_known_values.copy()
    for _ in range(num_future_periods):
        lag_1 = last_known_values[-1]
        rolling_mean_3 = np.mean(last_known_values[-3:])
        rolling_std_3 = np.std(last_known_values[-3:])
        rolling_mean_6 = np.mean(last_known_values[-6:])
        rolling_std_6 = np.std(last_known_values[-6:])
        future_features.append([lag_1, rolling_mean_3, rolling_std_3, rolling_mean_6, rolling_std_6])
        next_forecast = best_xgb_model.predict(np.array([future_features[-1]]))
        last_known_values = np.append(last_known_values, next_forecast)
    return np.array(future_features)

# Generate dynamic future features for 6 months
future_features = create_future_features(np.append(y_train.values, y_test.values), 6)

# Forecasting for the next 6 months using XGBoost
xgb_future_forecast = best_xgb_model.predict(future_features)

# Combine current and future forecasts
xgb_combined_forecast = np.concatenate([xgb_forecast, xgb_future_forecast])

# Combine dates for current and future forecasts
combined_dates = pd.concat([test['ds'], pd.Series(pd.date_range(start=test['ds'].max(), periods=7, freq='M')[1:])])

# Evaluate the improved XGBoost model performance
xgb_mae = mean_absolute_error(y_test, xgb_forecast)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_forecast))
xgb_mape = np.mean(np.abs((y_test - xgb_forecast) / y_test)) * 100
xgb_r2 = r2_score(y_test, xgb_forecast)

print(f'Improved XGBoost MAE: {xgb_mae}')
print(f'Improved XGBoost RMSE: {xgb_rmse}')
print(f'Improved XGBoost MAPE: {xgb_mape}')
print(f'Improved XGBoost R2: {xgb_r2}')



Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters: {'subsample': 0.9, 'n_estimators': 700, 'max_depth': 3, 'learning_rate': 0.03, 'colsample_bytree': 1.0}
Improved XGBoost MAE: 24298.452205882353
Improved XGBoost RMSE: 30644.130272680326
Improved XGBoost MAPE: 5.042070105703808
Improved XGBoost R2: 0.6644468864100024


In [10]:
# Plot XGBoost results
fig = make_subplots(rows=1, cols=1)

# Training data
fig.add_trace(go.Scatter(x=train['ds'], y=train['y'], mode='lines', name='Training Data', line=dict(color='royalblue', width=2)))

# Test data
fig.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Actual Data', line=dict(color='red', width=2)))

# XGBoost Forecast
fig.add_trace(go.Scatter(x=test['ds'], y=xgb_forecast, mode='lines', name='XGBoost Forecast', line=dict(dash='dot', color='green', width=2)))

# Customize layout
fig.update_layout(
    title={
        'text': 'XGBoost Model: Training, Test, and Forecast',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'font': dict(size=20, color='darkblue')
    },
    xaxis_title='Date',
    yaxis_title='Tourism Arrivals',
    yaxis=dict(
        tickformat='~s',  # Format y-axis labels to show "K" for thousands
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    xaxis=dict(
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    legend=dict(
        x=1.1,  # Position legend outside the chart
        y=1,
        xanchor='left',
        yanchor='top',
        traceorder='normal',
        font=dict(size=12),
        bgcolor='rgba(255, 255, 255, 0)'
    ),
    plot_bgcolor='white',  # Set plot background color
    paper_bgcolor='white',  # Set paper background color
    margin=dict(l=40, r=40, t=80, b=40)  # Adjust margins
)

fig.show()


### Combined Forecast and Visualization

In [11]:


# Combine predictions with weighted average
weight_arima = 0.4
weight_xgb = 0.6

combined_forecast = (weight_arima * arima_forecast_mean[:len(test)]) + (weight_xgb * xgb_forecast)
combined_future_forecast = (weight_arima * arima_future_forecast.values) + (weight_xgb * xgb_future_forecast)
combined_forecast_all = np.concatenate([combined_forecast, combined_future_forecast])

# Evaluate the combined model performance
combined_mae = mean_absolute_error(test['y'], combined_forecast)
combined_rmse = np.sqrt(mean_squared_error(test['y'], combined_forecast))
combined_mape = np.mean(np.abs((test['y'] - combined_forecast) / test['y'])) * 100
combined_r2 = r2_score(test['y'], combined_forecast)

print(f'Combined Model MAE: {combined_mae}')
print(f'Combined Model RMSE: {combined_rmse}')
print(f'Combined Model MAPE: {combined_mape}')
print(f'Combined Model R2: {combined_r2}')

# Create future dates for the next 6 months
future_dates = pd.date_range(start=test['ds'].max(), periods=7, freq='M')[1:]

# Plot the results using Plotly
fig = make_subplots(rows=1, cols=1)

# Training data
fig.add_trace(go.Scatter(x=train['ds'], y=y_train, mode='lines', name='Training Data', line=dict(color='royalblue', width=2)))

# Test data
fig.add_trace(go.Scatter(x=test['ds'], y=y_test, mode='lines', name='Actual Data', line=dict(color='red', width=2)))

# Combined Forecast
fig.add_trace(go.Scatter(x=combined_dates, y=combined_forecast_all, mode='lines', name='Forecast (ARIMA + XGBoost)', line=dict(dash='dashdot', color='purple', width=2)))

# Confidence Interval
fig.add_trace(go.Scatter(
    x=np.concatenate([test['ds'], future_dates, future_dates[::-1], test['ds'][::-1]]),
    y=np.concatenate([combined_forecast - combined_rmse, combined_future_forecast - combined_rmse, (combined_future_forecast + combined_rmse)[::-1], (combined_forecast + combined_rmse)[::-1]]),
    fill='toself',
    fillcolor='rgba(173, 216, 230, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='95% Confidence Interval'
))

# Customize layout
fig.update_layout(
    title={
        'text': 'Tourism Arrivals: Predictions for the Upcoming 6 Months',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'font': dict(size=20, color='darkblue')
    },
    xaxis_title='Date',
    yaxis_title='Tourism Arrivals',
    yaxis=dict(
        tickformat='~s',  # Format y-axis labels to show "K" for thousands
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    xaxis=dict(
        titlefont=dict(size=14),
        tickfont=dict(size=12),
        showgrid=True,  # Add grid lines
        gridcolor='lightgray'  # Set grid line color
    ),
    legend=dict(
        x=1.1,  # Position legend outside the chart
        y=1,
        xanchor='left',
        yanchor='top',
        traceorder='normal',
        font=dict(size=12),
        bgcolor='rgba(255, 255, 255, 0)'
    ),
    plot_bgcolor='white',  # Set plot background color
    paper_bgcolor='white',  # Set paper background color
    margin=dict(l=40, r=40, t=80, b=40)  # Adjust margins
)

fig.show()


Combined Model MAE: 21650.18200994424
Combined Model RMSE: 26229.365693615713
Combined Model MAPE: 4.5424324509271035
Combined Model R2: 0.7541658291903357


### Creating and Displaying the Table

In [18]:
from IPython.display import display

# Create a table with forecasted, actual (test), and error values
forecast_table = pd.DataFrame({
    'Date': test['ds'],
    'Actual': test['y'],
    'Forecasted': combined_forecast,
    'Error': test['y'] - combined_forecast
})

# Calculate the error metrics

forecast_table['MAPE'] = abs(forecast_table['Error'] / forecast_table['Actual']) * 100


# Display the table in a more visually appealing format
display(forecast_table)

,Date,Actual,Forecasted,Error,MAPE
120,2023-01-01,402943.0,378743.866384,24199.133616,6.005597
121,2023-02-01,434503.0,403494.949538,31008.050462,7.136441
122,2023-03-01,453093.0,432634.781404,20458.218596,4.515236
123,2023-04-01,430129.0,417142.861147,12986.138853,3.019127
124,2023-05-01,431650.0,415080.654810,16569.345190,3.838607
125,2023-06-01,416150.0,413120.265355,3029.734645,0.728039
126,2023-07-01,457412.0,461362.638840,-3950.638840,0.863694
127,2023-08-01,464637.0,455171.163768,9465.836232,2.037254
128,2023-09-01,424901.0,441907.723043,-17006.723043,4.002514
129,2023-10-01,480330.0,467514.515576,12815.484424,2.668058


In [19]:
forecast_table.to_csv('forecasted_value.csv', index=False)

In [18]:
future_forecast_table = pd.DataFrame({
    'Date': future_dates,
    'Forecasted': combined_future_forecast
})

# Display the table in a more visually appealing format
display(future_forecast_table)

,Date,Forecasted
0,2024-06-30,545320.078784
1,2024-07-31,556383.657721
2,2024-08-31,547784.206724
3,2024-09-30,527644.679937
4,2024-10-31,551170.305862
5,2024-11-30,553003.335555


In [19]:
#future_forecast_table.to_csv('future_forecast_value.csv', index=False)